In [1]:
from libs import *
from paths import *
from helper_functions import *
from pandas_datareader import data as pdr
from yahoo_fin import stock_info as si
import yfinance as yf
yf.pdr_override()

In [2]:
fcustom = Custom()
cols = [0,1,2,3,4,6,43,44,45,51,52,53,63,65,68]

vol1 = 'Month - Over 5%'
filters_dict1 = {'Market Cap.': '+Micro (over $50mln)', 'Volatility': vol1}
fcustom.set_filter(filters_dict=filters_dict1)
df1 = fcustom.screener_view(columns=cols, sleep_sec = 1, order= 'Performance (Month)', ascend=False)
print(df1.shape[0])

In [3]:
end_date = date.today()
start_date = end_date - timedelta(days=250)

In [4]:
today_ = datetime.today().strftime('%Y/%m/%d')

In [5]:
path_ = q_wl / today_ 
if not os.path.isdir(path_):
    os.mkdir(path_)
os.chdir(path_) 
if not os.path.isdir('temp'):
    os.mkdir('temp')
os.chdir('temp')

In [71]:
# IF CSV FILES PRESENT

# files = glob('*.csv')
# ticker_df = pd.DataFrame(columns=['Ticker', '1m', '3m', '6m', 'DV', 'ADR%'])

# for f in files:
#     df = pd.read_csv(f)
#     ticker = f.split('.')[0]
#     for i in [6]:
#         n_month_gain(df, i)
#     # print(ticker)
#     ticker_df.loc[len(ticker_df.index)] = [ticker, df['1M_low_gain'].iloc[-1], df['3M_low_gain'].iloc[-1], df['6M_low_gain'].iloc[-1],  df['DollarVolume'].iloc[-1], df['ADR%'].iloc[-1]]
#     df.to_csv(ticker + '.csv')


In [6]:
tickers = df1['Ticker']
request_times = []
errors = 0
ticker_df = pd.DataFrame(columns=['Ticker', '1m', '3m', '6m', 'DV', 'ADR%'])

In [8]:
start_time = time.time()
for ticker in tickers:
    try:
        # Download historical data as CSV for each stock (makes the process faster)
        while len(request_times) >= 2000 and (time.time() - request_times[-2000]) < 3600:
            time.sleep(1)
        
        df = yf.download(ticker, period='6mo')
        request_times.append(time.time())
        
        #pre-processing
        df.rename(columns=str.lower, inplace=True)
        df.drop('close', axis=1, inplace=True)
        df.rename(columns={'adj close': 'close'}, inplace=True)
        adr(df)
        df['DollarVolume'] = df['close'] * df['volume']
        
        #n-month gains
        for i in [1,3,6]:
            n_month_gain(df, i)
        # print(ticker_df)
        ticker_df.loc[len(ticker_df.index)] = [ticker, df['1M_low_gain'].iloc[-1], df['3M_low_gain'].iloc[-1], df['6M_low_gain'].iloc[-1],  df['DollarVolume'].iloc[-1], df['ADR%'].iloc[-1]]
        print(ticker, df.shape)
        df.to_csv(ticker + '.csv')
    except:
        print('Error for ticker: ', ticker)
        errors += 1
print('Time taken: {}'.format(time.time() - start_time))

[*********************100%***********************]  1 of 1 completed
GNS (127, 13)
[*********************100%***********************]  1 of 1 completed
BBAI (127, 13)
[*********************100%***********************]  1 of 1 completed
HPCO (113, 13)
[*********************100%***********************]  1 of 1 completed
SOUN (127, 13)
[*********************100%***********************]  1 of 1 completed
MSGM (127, 13)
[*********************100%***********************]  1 of 1 completed
CVNA (127, 13)
[*********************100%***********************]  1 of 1 completed
GFAI (127, 13)
[*********************100%***********************]  1 of 1 completed
IPDN (127, 13)
[*********************100%***********************]  1 of 1 completed
SLQT (127, 13)
[*********************100%***********************]  1 of 1 completed
INBS (127, 13)
[*********************100%***********************]  1 of 1 completed
BSGM (127, 13)
[*********************100%***********************]  1 of 1 completed
DGHI (12

In [9]:
ticker_df = pd.merge(ticker_df, df1, on='Ticker')
ticker_df = ticker_df[['Ticker', '1m','3m','6m','DV', 'ADR%', 'Company', 'Sector', 'Industry', 'Market Cap', 'Price', 'Earnings']]

In [10]:
#PARAMETERS
mil = 10 ** 6
adr_filter = 5.0
dv_filter = 1 * mil
limit = 50

In [11]:
#FILTER
ticker_df_filt = ticker_df[(ticker_df['ADR%'] >= adr_filter) & (ticker_df['DV'] >= dv_filter)]

In [12]:
#RANK
ticker_df_filt['1m_Rating'] = ticker_df_filt['1m'].rank(ascending=False)
ticker_df_filt['3m_Rating'] = ticker_df_filt['3m'].rank(ascending=False)
ticker_df_filt['6m_Rating'] = ticker_df_filt['6m'].rank(ascending=False)

ticker_df_1m = ticker_df_filt[ticker_df_filt['1m_Rating'] <= limit]
ticker_df_3m = ticker_df_filt[ticker_df_filt['3m_Rating'] <= limit]
ticker_df_6m = ticker_df_filt[ticker_df_filt['6m_Rating'] <= limit]

In [13]:
ticker_df_final = pd.concat([ticker_df_1m, ticker_df_3m, ticker_df_6m], axis=0)
ticker_df_final = ticker_df_final.drop_duplicates(subset=['Ticker'], keep='first')

In [14]:
os.chdir(q_wl / today_)

In [15]:
s1 = set(ticker_df_1m['Ticker'])
set_to_tv(s1, datetime.today().strftime('%Y%m%d') + '_1_M_Q_US.txt')
s3 = set(ticker_df_3m['Ticker'])
set_to_tv(s3, datetime.today().strftime('%Y%m%d') + '_3_M_Q_US.txt')
s6 = set(ticker_df_6m['Ticker'])
set_to_tv(s6, datetime.today().strftime('%Y%m%d') + '_6_M_Q_US.txt')

s = set(ticker_df_final['Ticker'])
set_to_tv(s, datetime.today().strftime('%Y%m%d') + '_Q_US.txt')


20230210_1_M_Q_US.txt
20230210_3_M_Q_US.txt
20230210_6_M_Q_US.txt
20230210_Q_US.txt


In [16]:
# ind = fcustom.get_filter_options('Industry')
# sector = set(df1['Sector'])

In [17]:
ticker_df_final.to_csv('INFO.csv')

In [93]:
shutil.rmtree('temp')